### Preprocess Perturb-seq data

In [71]:
import pandas as pd
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl

data_path = '/data_volume/memento/method_comparison/tfko140/'
### Read single cell and bulk data

adata = sc.read(data_path + 'tfko.sng.guides.full.ct.h5ad')

Only considering the two last: ['.ct', '.h5ad'].
Only considering the two last: ['.ct', '.h5ad'].


In [72]:
wt_guides = adata.obs.query('WT=="T"')['guide1_cov'].drop_duplicates().tolist()

In [73]:
guides = adata.obs[['guide1_cov']].drop_duplicates().reset_index(drop=True)\
    .query('guide1_cov not in @wt_guides')
guides['target'] = guides['guide1_cov'].str.split('.').str[0]
guides = guides.groupby('target').agg({'guide1_cov':[list, 'count']})
guides.columns=['list', 'count']
guides = guides.query('count > 1')
guides = dict(zip(guides.index, guides['list']))

In [74]:
guides2 = adata.obs[['guide1_cov']].drop_duplicates().reset_index(drop=True)\
    .query('guide1_cov not in @wt_guides')
guides2['target'] = guides2['guide1_cov'].str.split('.').str[0]

guides2 = guides2.query('target in @guides')
guides2.to_csv('guides.csv', index=False)


In [75]:
with open('guide_pairs.pkl', 'wb') as f:
    pkl.dump(guides, f)
    

In [ ]:
inds = list(set(adata.obs.donor))

for target, guide_pair in guides.items():

    for guide in guide_pair:

        subset = adata[(adata.obs['WT'] == 'T') | (adata.obs['guide1_cov']==guide)]
        pseudobulks = []
        names = []
        adata_list = []
        for condition in ['T', 'F']:
            for ind in inds:
                condition_ind_adata = subset[(subset.obs['donor']==ind) & (subset.obs['WT']==condition)].copy()
                print('processing', target, guide, condition, ind, 'num cells:', condition_ind_adata.shape[0])
                adata_list.append(condition_ind_adata.copy())
                pseudobulks.append( condition_ind_adata.X.sum(axis=0).A1)
                names.append('_'.join([target, guide, condition, ind]) )
        sc_data = sc.AnnData.concatenate(*adata_list)
        pseudobulks = np.vstack(pseudobulks)
        pseudobulks = pd.DataFrame(pseudobulks.T, columns=names, index=subset.var.index.tolist())

        pseudobulks.to_csv(data_path + 'tfko140.pseudobulk.{}.{}.csv'.format(target, guide))

        sc_data.write(data_path + 'tfko140.single_cell.{}.{}.h5ad'.format(target, guide))

In [4]:
adata.obs.head(5)

,concat_index,donor,multiplet,well,guide1_cov,guide2_cov,guide3_cov,guide4_cov,guide1_ce_cov,guide2_ce_cov,...,guide_multiplet_cov,percent_mito,percent_ribo,percent_hist,numi,n_genes,ct,leiden,annotation,WT
index,,,,,,,,,,,,,,,,,,,,,
GATCGTATCCAGAAGG-3,0,IGTB884,SNG,3,FOSL2.28412051.AGTGTGCAAGATTAGCCCCG,SMAD2.47896526.CTAAATGTGTTACCATACCA,nan,nan,0.036424,0.146533,...,2,0.054186,0.275146,0.001947,3082.0,1270,naive,0,0,F
CTTTGCGGTCAACATC-8,0,IGTB1982,SNG,8,RC3H1.173961818.CTGGATTCTGGTACATACGA,nan,nan,nan,0.154435,NaN,...,1,0.030713,0.359974,0.001321,3028.0,1134,naive,0,0,F
ATCATGGCAGTGGAGT-7,0,IGTB884,SNG,7,SLC30A9.42063117.AAAGAGGATTACCCATAGAA,TFB2M.246565843.CTGGATTGCACTCCAGCAGT,nan,nan,0.097216,0.165600,...,2,0.041566,0.357315,0.009451,10369.0,2529,prolif_1,1,1,F
TGGCGCAAGGATGCGT-6,0,IGTB1762,SNG,6,MAFK.1540041.CCTGCGGGGTCTCACCAAGG,nan,nan,nan,0.005697,NaN,...,1,0.059738,0.335797,0.001540,6495.0,1923,naive,0,0,F
CGAGCACGTAGAGCTG-7,0,IGTB2065,SNG,7,SRA1.140551987.AGACTGCCGTGGCCACACAA,nan,nan,nan,0.109404,NaN,...,1,0.030303,0.428344,0.000911,4389.0,1288,naive,0,0,F


In [ ]:
target_genes = 

In [3]:
adata

AnnData object with n_obs × n_vars = 173018 × 14988
    obs: 'concat_index', 'donor', 'multiplet', 'well', 'guide1_cov', 'guide2_cov', 'guide3_cov', 'guide4_cov', 'guide1_ce_cov', 'guide2_ce_cov', 'guide3_ce_cov', 'guide4_ce_cov', 'guide_multiplet_cov', 'percent_mito', 'percent_ribo', 'percent_hist', 'numi', 'n_genes', 'ct', 'leiden', 'annotation', 'WT'
    var: 'gene_ids-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0', 'gene_ids-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0', 'gene_ids-1-0-0-0-0-0-0-0-0-0-0-0-0-0', 'gene_ids-1-0-0-0-0-0-0-0-0-0-0-0-0', 'gene_ids-1-0-0-0-0-0-0-0-0-0-0-0', 'gene_ids-1-0-0-0-0-0-0-0-0-0-0', 'gene_ids-1-0-0-0-0-0-0-0-0-0', 'gene_ids-1-0-0-0-0-0-0-0-0', 'gene_ids-1-0-0-0-0-0-0-0', 'gene_ids-1-0-0-0-0-0-0', 'gene_ids-1-0-0-0-0-0', 'gene_ids-1-0-0-0-0', 'gene_ids-1-0-0-0', 'gene_ids-1-0-0', 'gene_ids-1-0', 'gene_ids-1', 'n_cells'